In [1]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data

pd.read_pickle(r'.\Grass_Today')

,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Elo_Fav_Elo,Elo_Dog_Elo,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc
0,01:00,1.56,2.42,Cameron Norrie,Mattia Bellucci,67%,3,Mattia Bellucci,57%,7,...,1649.330210,1564.040483,0.6,0.6,0.81,0.27,0.00,0.00,0.6,0.80
1,01:10,1.71,2.14,Jordan Thompson,Jordan Thompson,56%,9,Luciano Darderi,33%,6,...,1631.237186,1488.843315,0.6,0.3,0.77,0.16,0.00,0.00,0.6,0.40
2,01:45,1.81,2.00,Arthur Rinderknech,Arthur Rinderknech,60%,10,Kamil Majchrzak,100%,2,...,1662.950422,1527.477402,0.5,1.0,0.84,0.14,0.81,0.22,0.6,0.50
3,01:50,1.43,2.82,Joao Fonseca,Nicolas Jarry,50%,2,Nicolas Jarry,83%,6,...,1690.689043,1628.217645,0.3,0.7,NaN,NaN,0.87,0.05,0.6,1.00
4,02:00,1.42,2.87,Karen Khachanov,Karen Khachanov,57%,7,Nuno Borges,25%,4,...,1728.189624,1528.675666,0.7,0.2,0.82,0.20,0.84,0.13,0.8,0.60
5,03:15,1.04,12.44,Carlos Alcaraz,Carlos Alcaraz,100%,3,Jan Lennard Struff,56%,9,...,2037.996847,1672.741191,0.9,0.6,0.84,0.31,NaN,NaN,1.0,0.60
6,03:55,1.59,2.35,Elina Svitolina,Elise Mertens,67%,9,Elise Mertens,50%,2,...,1757.890140,1757.655526,0.7,0.9,0.64,0.37,0.68,0.39,0.6,0.80
7,06:05,1.25,3.96,Aryna Sabalenka,Aryna Sabalenka,86%,7,Emma Raducanu,67%,6,...,1823.632766,1714.818792,0.9,0.7,0.68,0.37,0.68,0.39,0.8,0.60
8,21:00,1.41,2.92,Liudmila Samsonova,Liudmila Samsonova,60%,10,Daria Kasatkina,50%,8,...,1756.752495,1622.660209,0.7,0.5,0.81,0.31,0.61,0.47,0.8,0.40
9,21:00,1.46,2.70,Dayana Yastremska,Dayana Yastremska,70%,10,Jessica Bouzas Maneiro,100%,3,...,1759.419136,1534.832288,0.7,1.0,0.65,0.28,0.00,0.00,0.6,0.75


In [107]:
clay_womens=hist_data_dog('Clay','Womens',1,2,datefilter)
clay_mens=hist_data_dog('Clay','Mens',1,2,datefilter)
temp=pd.concat([clay_womens,clay_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()
temp['Fav_Win_Count'] = temp.groupby('Fav')['Fav_Win'].cumsum()

In [108]:
temp
date_filter=temp[temp['Date']>'2024-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']<=1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Fav_Odds']>=1]
#oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>0]
#oddsfilter=oddsfilter[oddsfilter['Fav_Win_Count']>0]
#oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
oddsfilter=oddsfilter[oddsfilter['Dog_Top100']==0]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']>.5]
oddsfilter=oddsfilter[oddsfilter['dog_rank']>100]
oddsfilter=oddsfilter[oddsfilter['dog_rank_high']>100]
oddsfilter=oddsfilter[oddsfilter['fav_rank']<100]
oddsfilter=oddsfilter[oddsfilter['Sex']!='Womvens']

oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] != row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 196.0 178 0.8202247191011236


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Fav_Win_Count,Profit
64,2024-02-22,4.15,Joao Fonseca,1.23,4.15,Arthur Fils,Arthur Fils,11,Joao Fonseca,5,...,34.0,636.0,False,True,1,2,1,1,0,-100.0
163,2024-03-13,4.4,Daniel Michalski,1.19,4.40,Dominic Thiem,Dominic Thiem,10,Daniel Michalski,10,...,3.0,243.0,False,True,1,1,1,2,0,-100.0
226,2024-03-20,1.19,Juan Pablo Varillas,1.19,4.39,Juan Pablo Varillas,Juan Pablo Varillas,15,Carlos Prado Angelo Juan,5,...,60.0,559.0,True,False,0,1,0,0,5,19.0
259,2024-03-22,1.09,Juan Pablo Varillas,1.09,6.84,Juan Pablo Varillas,Juan Pablo Varillas,10,Nicolas Zanellato,10,...,60.0,469.0,True,False,0,1,0,1,6,9.0
272,2024-03-23,1.19,Juan Pablo Varillas,1.19,4.48,Juan Pablo Varillas,Juan Pablo Varillas,10,Agustin Gomez Federico,10,...,60.0,342.0,True,False,0,1,0,2,7,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6773,2025-05-02,3.72,Hady Habib,1.25,3.72,Vit Kopriva,Vit Kopriva,30,Hady Habib,10,...,95.0,166.0,False,True,1,13,1,7,17,-100.0
6854,2025-05-05,1.24,Olivia Gadecki,1.24,3.94,Olivia Gadecki,Nicole Fossa Huergo,12,Nicole Fossa Huergo,9,...,83.0,256.0,True,False,0,0,0,1,3,24.0
6877,2025-05-06,1.33,Lulu Sun,1.33,3.32,Lulu Sun,Lulu Sun,7,Georgia Pedone,7,...,39.0,183.0,True,False,0,1,0,0,1,33.0
6942,2025-05-07,1.2,Peyton Stearns,1.20,4.56,Peyton Stearns,Peyton Stearns,10,Nuria Brancaccio,10,...,42.0,167.0,True,False,0,5,0,1,7,20.0


In [109]:
temp
date_filter=temp[temp['Date']>'2024-01-01']
oddsfilter=date_filter[date_filter['Fav_Odds']<=1.4].copy()
oddsfilter=oddsfilter[oddsfilter['Fav_Odds']>=1]
oddsfilter=oddsfilter[oddsfilter['fav_percent']>0.5]
oddsfilter=oddsfilter[oddsfilter['dog_percent']<0.5]
oddsfilter=oddsfilter[oddsfilter['dog_rank']>200]
oddsfilter=oddsfilter[oddsfilter['dog_rank_high']>100]
oddsfilter=oddsfilter[oddsfilter['fav_rank']>200]
oddsfilter=oddsfilter[oddsfilter['fav_rank_high']<150]
oddsfilter=oddsfilter[oddsfilter['Dog_Top100']==0]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']<10]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']<5]


oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Fav_Odds'] - 1) * stake if row['Winner'] != row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Fav']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter

Total Profit: 48.99999999999993 83 0.8072289156626506


,Date,Winner_Odds,Winner,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Games,Dog,Dog_Games,...,fav_rank_high,dog_rank_high,Fav_Win,DogWin,Fav_Loss,Fav_Loss_Count,Dog_Win,Dog_Win_Count,Fav_Win_Count,Profit
240,2024-03-20,2.79,Sandro Kopp,1.40,2.79,Francesco Maestrelli,Francesco Maestrelli,10,Sandro Kopp,9,...,149.0,386.0,False,True,1,2,1,1,1,-100.0
284,2024-03-24,1.22,Mathias Bourgue,1.22,4.06,Mathias Bourgue,Mathias Bourgue,10,Lorenzo Rottoli,10,...,140.0,403.0,True,False,0,1,0,0,2,22.0
367,2024-03-28,3.26,Goncalo Oliveira,1.32,3.26,Gianluca Mager,Gianluca Mager,15,Goncalo Oliveira,5,...,62.0,194.0,False,True,1,3,1,1,4,-100.0
453,2024-04-02,1.32,Gianluca Mager,1.32,3.28,Gianluca Mager,Gianluca Mager,10,Gonzalo Bueno,11,...,62.0,309.0,True,False,0,3,0,0,5,32.0
789,2024-04-14,1.16,Chun Hsin Tseng,1.16,4.18,Chun Hsin Tseng,Chun Hsin Tseng,10,Joao Domingues,9,...,83.0,150.0,True,False,0,1,0,0,4,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6604,2025-04-29,1.26,Giulio Zeppieri,1.26,3.64,Giulio Zeppieri,Giulio Zeppieri,10,Tiago Pereira,11,...,110.0,409.0,True,False,0,8,0,4,7,26.0
6681,2025-04-30,3.63,Conner Huertas Del Pino,1.25,3.63,Nicolas Kicker,Nicolas Kicker,10,Conner Huertas Del Pino,11,...,78.0,520.0,False,True,1,3,1,4,5,-100.0
6802,2025-05-04,3.25,Maxime Chazal,1.31,3.25,Benjamin Hassan,Benjamin Hassan,11,Maxime Chazal,13,...,143.0,234.0,False,True,1,7,1,4,13,-100.0
6881,2025-05-06,1.35,Nicolas Kicker,1.35,3.00,Nicolas Kicker,Nicolas Kicker,10,Joaquin Rodriguez Lorenzo,10,...,78.0,439.0,True,False,0,3,0,1,6,35.0


In [110]:
date_filter=pd.read_pickle('.\Clay_Today')
oddsfilter=date_filter[date_filter['Fav_Odds']<1.3].copy()

oddsfilter=oddsfilter[oddsfilter['Fav_Odds']>=1]

oddsfilter=oddsfilter[oddsfilter['Sex']!='Womvens']
oddsfilter

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_3636\809562928.py:1: SyntaxWarning: invalid escape sequence '\C'
  date_filter=pd.read_pickle('.\Clay_Today')


,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,...,Fav_Top100,Dog_Top100,Fav_Serve%,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count
0,00:10,1.04,11.94,Mirra Andreeva,Mirra Andreeva,87%,15,Emiliana Arango,0%,2,...,0.7,0.4,0.56,0.21,0.00,0.00,0.6,0.4,7.0,5.0
2,00:15,1.26,3.65,Filip Misolic,Filip Misolic,100%,1,Mees Rottgering,100%,1,...,0.2,0.0,0.71,0.27,NaN,NaN,0.6,1.0,11.0,2.0
8,02:00,1.12,6.27,Emma Navarro,Emma Navarro,61%,18,Kamilla Rakhimova,29%,7,...,0.4,0.4,0.51,0.29,0.60,0.33,0.4,0.6,10.0,10.0
14,03:45,1.25,3.96,Jack Draper,Luciano Darderi,47%,17,Luciano Darderi,44%,27,...,0.5,0.6,0.85,0.24,0.00,0.00,0.8,0.6,4.0,1.0
15,04:15,1.14,5.68,Elena Rybakina,Elena Rybakina,87%,15,Eva Lys,25%,4,...,0.8,0.4,0.78,0.30,0.50,0.39,0.6,0.4,2.0,4.0
18,06:15,1.08,7.81,Alexander Zverev,Alexander Zverev,79%,47,Camilo Ugo Carabelli,25%,8,...,0.7,0.5,0.83,0.26,0.65,0.21,0.8,0.6,11.0,16.0
24,20:00,1.25,3.94,Tommy Paul,Tommy Paul,67%,15,Roberto Bautista Agut,54%,13,...,0.6,0.5,0.80,0.24,0.82,0.27,0.6,0.4,3.0,4.0
27,21:30,1.15,5.31,Taylor Fritz,Taylor Fritz,62%,26,Marcos Giron,27%,11,...,0.6,0.3,0.84,0.24,0.77,0.17,0.6,0.4,10.0,6.0
30,21:30,1.29,3.57,Sebastian Korda,Sebastian Korda,67%,6,Aleksandar Vukic,22%,9,...,0.4,0.3,0.80,0.26,0.85,0.13,0.4,0.8,10.0,5.0
31,21:30,1.18,4.84,Casper Ruud,Casper Ruud,81%,37,Alexander Bublik,33%,12,...,0.7,0.5,0.85,0.23,0.81,0.20,1.0,0.8,4.0,4.0


In [111]:
data=pd.read_sql_query(f"Select distinct * from results_clay_1 where date > '2024-01-01' ",con=devengine)#[['Fav','Dog','Winner','Fav_Odds']]
data["Fav_Odds"] = data.Fav_Odds.astype(float)
data["Dog_Odds"] = data.Dog_Odds.astype(float)
data['Fav_Win'] = data['Winner']==data['Fav']
data['DogWin'] = data['Winner']!=data['Fav']
data=data[(data['Fav_Odds']>=1)&(data['Fav_Odds']<=1.3)]
data[['Fav','Dog','Fav_Odds','Fav_Win']]
results = data.groupby('Fav').agg(
    Games_Played=('Fav_Win', 'count'),
    Win_Percentage=('Fav_Win', 'mean')
).reset_index()
results=results[results['Games_Played']>5]
results=results[results['Win_Percentage']>.9]
results.sort_values(by=['Win_Percentage','Games_Played'],ascending=False)

,Fav,Games_Played,Win_Percentage
269,Mayar Sherif,19,1.000000
117,Emma Navarro,14,1.000000
201,Juan Bautista Torres,11,1.000000
297,Olga Danilovic,10,1.000000
214,Katie Volynets,8,1.000000
226,Lorenzo Musetti,8,1.000000
265,Matteo Berrettini,8,1.000000
25,Andrea Collarini,7,1.000000
167,Holger Rune,7,1.000000
376,Zizou Bergs,7,1.000000


In [112]:
data=pd.read_sql_query(f"Select distinct * from results_hard_1 where date > '2024-01-01' ",con=devengine)#[['Fav','Dog','Winner','Fav_Odds']]
data["Fav_Odds"] = data.Fav_Odds.astype(float)
data["Dog_Odds"] = data.Dog_Odds.astype(float)
data['Fav_Win'] = data['Winner']==data['Fav']
data['DogWin'] = data['Winner']!=data['Fav']
data=data[(data['Fav_Odds']>=1)&(data['Fav_Odds']<=1.3)]
data[['Fav','Dog','Fav_Odds','Fav_Win']]
results = data.groupby('Fav').agg(
    Games_Played=('Fav_Win', 'count'),
    Win_Percentage=('Fav_Win', 'mean')
).reset_index()
results=results[results['Games_Played']>5]
results=results[results['Win_Percentage']>.9]
results.sort_values(by=['Win_Percentage','Games_Played'],ascending=False)


,Fav,Games_Played,Win_Percentage
314,Paula Badosa,15,1.000000
218,Karolina Muchova,13,1.000000
191,James Duckworth,12,1.000000
284,Mattia Bellucci,11,1.000000
155,Gabriel Diallo,10,1.000000
310,Pablo Carreno Busta,10,1.000000
72,Brandon Holt,9,1.000000
152,Frances Tiafoe,8,1.000000
246,Lloyd Harris,8,1.000000
251,Lucas Pouille,8,1.000000


In [115]:
data=pd.read_sql_query(f"Select distinct * from AllMatches where date > '2023-01-01' ",con=devengine)#[['Fav','Dog','Winner','Fav_Odds']]
data["Fav_Odds"] = data[['Player_1_Odds','Player_2_Odds']].min(axis=1)
data["Dog_Odds"] = data[['Player_1_Odds','Player_2_Odds']].max(axis=1)
data['Fav_Win'] = data['Player_1_Odds']==data['Fav_Odds']
data['Fav']=data.apply(lambda x: x['Player_1'] if x['Player_1_Odds']< x['Player_2_Odds'] else x['Player_2'],axis=1 )
data=data[(data['Fav_Odds']>=1.2)&(data['Fav_Odds']<=1.3)]
data=data[data['Surface']=='Clay']
#data[['Fav','Dog','Fav_Odds','Fav_Win']]
results = data.groupby('Fav').agg(
    Games_Played=('Fav_Win', 'count'),
    Win_Percentage=('Fav_Win', 'mean')
).reset_index()
results=results[results['Games_Played']>0]
results=results[results['Win_Percentage']>.1]
results.sort_values(by=['Win_Percentage','Games_Played'],ascending=False)
results[results['Fav']=='Elina Svitolina']#.to_clipboard()

,Fav,Games_Played,Win_Percentage
195,Elina Svitolina,7,0.857143


In [114]:
pd.read_sql_query(f"Select distinct * from AllMatches where date > '2023-01-01' ",con=devengine)#[['Fav','Dog','Winner','Fav_Odds']]

,level_0,index,Date,Sex,Tournament,Player_1,Player_2,Player_1_Odds,Player_2_Odds,Surface,...,w1,w2,w3,w4,w5,l1,l2,l3,l4,l5
0,0.0,0.0,2023-01-02,Mens,Oeiras,Alibek Kachmazov,Timo Legout,1.41,2.72,Hard,...,6,6,,,,2,1,,,
1,1.0,1.0,2023-01-02,Mens,Oeiras,Rudolf Molleker,Simon Beaupain,1.30,3.28,Hard,...,6,6,,,,4,3,,,
2,2.0,2.0,2023-01-02,Mens,Oeiras,Joris De Loore,Oscar Jose Gutierrez,1.22,3.89,Hard,...,6,6,,,,1,2,,,
3,3.0,3.0,2023-01-02,Mens,Oeiras,Sebastian Fanselow,Fabio Coelho,1.16,4.48,Hard,...,6,7,,,,3,5,,,
4,4.0,4.0,2023-01-02,Mens,Oeiras,Edan Leshem,Henrique Rocha,1.35,2.94,Hard,...,0,7,6,,,6,5,2,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42744,NaN,NaN,2025-05-09,Womens,Rome WTA0,Marie Bouzkova,Beatriz Haddad Maia,1.82,1.96,Clay,...,6,6,,,,0,3,,,
42745,NaN,NaN,2025-05-09,Womens,Rome WTA0,Madison Keys,Varvara Gracheva,1.17,5.09,Clay,...,7,6,,,,6,1,,,
42746,NaN,NaN,2025-05-09,Womens,Rome WTA0,Elina Svitolina,Jessica Bouzas Maneiro,1.13,6.04,Clay,...,6,7,,,,4,6,,,
42747,NaN,NaN,2025-05-09,Womens,Rome WTA0,Jessica Pegula,Ashlyn Krueger,1.35,3.16,Clay,...,6,6,,,,4,2,,,
